In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from typing import Dict, Tuple
import re
import keras.layers as l
from keras import models, callbacks, utils, losses

In [5]:
text = ''
with open('harry_potter.txt', 'r') as file:
    text = file.read()

def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
alphabet = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)


In [6]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.SimpleRNN(128, return_sequences=True, stateful=True),
    l.Dense(len(alphabet) / 2, activation='relu'),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=50, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/50
6/6 [==============================] - 1s 60ms/step - loss: 8.1396 - accuracy: 0.0141
Epoch 2/50
6/6 [==============================] - 0s 60ms/step - loss: 7.6717 - accuracy: 0.0239
Epoch 3/50
6/6 [==============================] - 0s 59ms/step - loss: 7.3871 - accuracy: 0.0215
Epoch 4/50
6/6 [==============================] - 0s 57ms/step - loss: 7.3807 - accuracy: 0.0239
Epoch 5/50
6/6 [==============================] - 0s 60ms/step - loss: 7.3629 - accuracy: 0.0239
Epoch 6/50
6/6 [==============================] - 0s 59ms/step - loss: 7.3576 - accuracy: 0.0239
Epoch 7/50
6/6 [==============================] - 0s 58ms/step - loss: 7.3426 - accuracy: 0.0239
Epoch 8/50
6/6 [==============================] - 0s 58ms/step - loss: 7.3357 - accuracy: 0.0239
Epoch 9/50
6/6 [==============================] - 0s 60ms/step - loss: 7.3212 - accuracy: 0.0239
Epoch 10/50
6/6 [==============================] - 0s 58ms/step - loss: 7.3042 - accuracy: 0.0239
Epoch 11/50
6/6 [============

In [7]:
def predict_next(sample: str, model: keras.Sequential, tokenizer: Dict[str, int], vocabulary: Dict[int, str], n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [9]:
print(predict_next(
    sample='Гарри',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Гарри лунном свете блестит полоска серебрападают тёмные одежды кровь льётся литрами, и слышен крик Все стены до последнего дюйма заняты книжными


In [10]:
print(predict_next(
    sample='Магия',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Магия шкафами Гарри каждом мне три поднимать книги с одной профессор, с читаю Она мама Каким образом хранилище взгляд ещё их


In [12]:
print(predict_next(
    sample='МакГонагалл',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

МакГонагалл себе часть Ты Его были все Гарри, но же не Что к ответила за было бы стояли нам ничего все
